In [1]:
import numpy as np
import os
import sys
from itertools import islice # slicing for iterators
import math

from nltk.util import ngrams
from collections import Counter
from scipy.stats import pearsonr

In [2]:
def word_matches(h, ref):
    return sum(1 for w in h if w in ref)

In [3]:
def modified_precision(h,ref,n):
    ng_counts_h = Counter(ngrams(h,n))
    ng_counts_ref = Counter(ngrams(ref,n))
    modified_counts = Counter()
    if not ng_counts_h:
        return 0
    for ng in ng_counts_h.keys():
        modified_counts[ng] = max(modified_counts[ng], ng_counts_ref[ng])
        truncated_cts = Counter((ng, min(ng_counts_h[ng],modified_counts[ng])) for ng in ng_counts_h)
    return sum(truncated_cts.values())/float(sum(ng_counts_h.values()))

In [4]:
def weighted_F1_pos(h, ref, rho):
    mod_prec = modified_precision(h ,ref, 2)
    
    h = Counter(h)
    ref = Counter(ref)
    
    precision = {pos:0 for pos in h}
    for pos in h:
        if pos in ref:
            precision[pos] += 1
    
    for pos in ref:
        if pos in precision:
            precision[pos] = float(precision[pos]) / float(ref[pos])
    
    precision = 0.5 * sum(precision.values()) + 0.5 * mod_prec
    
    recall = {pos:0 for pos in ref}
    for pos in ref:
        if pos in h:
            recall[pos] += 1
    
    for pos in h:
        if pos in recall:
            recall[pos] = float(recall[pos]) / float(h[pos])
    
    recall = sum(recall.values())
    
    if precision == 0 or recall == 0:
        return 0
    f_1 = (1-rho) * precision ** -1 + rho * recall ** -1
    return f_1 ** -1

In [5]:
def weighted_F1_norm(h, ref, rho):
    
    mod_prec = modified_precision(h, ref, 2)
    h = Counter(h)
    ref = Counter(ref)
    
    precision = 0
    for word in h:
        if word in ref:
            precision += min(h[word], ref[word]) / float(h[word])
    
    precision = 0.5 * (precision / float(len(h))) + 0.5 * mod_prec
    
    recall = 0
    for word in ref:
        if word in h:
            recall += min(h[word], ref[word]) / float(ref[word])
    
    recall = recall / float(len(ref))
    
    if precision == 0 or recall == 0:
        return 0
    f_1 = (1-rho) * precision ** -1 + rho * recall ** -1
    return f_1 ** -1

In [6]:
def weighted_F1(h,ref,rho):
    #h = list(ngrams(h_i,2)) + list(h_i)
    #ref = list(ngrams(ref_i,2)) + list(ref_i)
    
    precision = float(sum(1 for w in h if w in ref))/len(h)
    precision = 0.5*modified_precision(h,ref,2) + 0.5*precision
    
    recall = float(sum(1 for w in ref if w in h))/len(ref)
    if precision == 0 or recall == 0:
        return 0
    ret = (1-rho) * (precision)**-1 + rho * (recall)**-1
    return  ret**-1

In [7]:
def BLEU(h,ref,n):
    def brevity_penalty(h,ref):
        c = len(h)
        r = len(ref)
        if c >= r:
            return 1
        else:
            return math.exp(1 - r/c)
        
        weights = [1.0/n for x in xrange(n)]
        mod_p = 0
        for i,w in enumerate(weights):
            x = modified_precision(h,ref,i)
            if x == 0:
                return 0
            mod_p += w*math.log(x)
            
        return brevity_penalty(h,ref) * math.exp(mod_p)

In [8]:
path_to_train = 'data/train-test.hyp1-hyp2-ref'

In [9]:
lines = [line.strip().split('|||') for line in open(path_to_train, 'r')]

In [10]:
path_to_tree_features = 'data/features.parse'
path_to_embedding_features = 'data/features.embedding.final'
path_to_lm_features = 'data/features.lm'
path_to_pos_features = 'data/features.pos'

In [11]:
path_to_labels = 'data/train.gold'

In [12]:
path_to_pos_tags = 'data/train-test.hyp1-hyp2-ref.pos'
pos_lines = [line.strip().split() for line in open(path_to_pos_tags, 'r')]
pos_lines = [[x.split('_')[1] for x in line] for line in pos_lines]
pos_tags = []
for i in range(0, len(pos_lines), 3):
    pos_tags.append(pos_lines[i:i+3])

In [13]:
f1_features = []
for line in lines:
    f1_features.append(weighted_F1_norm(line[0].split(), line[2].split(), 0.6) - weighted_F1_norm(line[1].split(), line[2].split(), 0.6))

In [14]:
pos_features = []
for line in pos_tags:
    pos_features.append(weighted_F1_pos(line[0], line[2], 0.6) - weighted_F1_pos(line[1], line[2], 0.6))

In [80]:
labels = [int(line.strip()) for line in open(path_to_labels)]

In [131]:
pearsonr(f1_features[:len(labels)], labels)

(0.21330695003500347, 2.3350437768947557e-267)

In [135]:
f1_features = [-x for x in f1_features] # I accidentally did 0 - 2 instead of 2 - 0
f1_preds = [-1 if x < 0 else 1 for x in f1_features]

In [144]:
pos_features = [-x for x in pos_features] # I accidentally did 0 - 2 instead of 2 - 0
pos_preds = [-1 if x < 0 else 1 for x in pos_features]

In [155]:
pearsonr(pos_features[:len(labels)], labels)

(0.12755806750468335, 1.7101038788232369e-95)

In [156]:
f1_f = open('data/features.f1', 'w')
pos_f = open('data/features.pos', 'w')
for f1, pos in zip(f1_features, pos_features):
    f1_f.write(str(f1) + '\n')
    pos_f.write(str(pos) + '\n')
f1_f.close()
pos_f.close()